<a href="https://colab.research.google.com/github/esmeantonio/BTTAI/blob/main/BTTAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
if not os.path.exists('/gd'):
    drive.mount('/gd')

In [ ]:
!pip install sentencepiece transformers==4.33 datasets sacremoses sacrebleu -q

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#TODO: Experiment with different ways of preprocessing data
df = pd.read_csv("Copy of Luganda (2).csv", encoding="ISO-8859-1")
df = df.loc[0:15021]
df["English"] = df["English"].str.lower() #Look at this line, consider whether it is needed or not
df["Luganda"] = df["Luganda"].str.lower()
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.dropna()
print(len(df)/2)

#TODO: Change the test-train-validation split
df['split'] = ["train" for i in range(7510)] + ["test" for i in range(7500)] + ["dev" for i in range(11)]
df.sample(10)

7510.5


,English,Luganda,split
8891,my request for more money was considered.,okusaba kwange okw'okufuna ensimbi endala kway...,test
2275,all categories of people are allowed into heal...,ebika by'abantu byonna bikirizibwa awajjanjabi...,train
12263,the disease can attack all ages.,obulwadde busobola okukwata emyaka gyonna.,test
6293,men still consider themselves superior to women.,abasajja bakyetwalira waggulu ku bakazi.,train
6311,people from poor backgrounds tend to be more s...,abantu abava mu maka amaavu batera okussaayo o...,train
10430,why are disciplinary actions important?,lwaki ebikolwa eby'okugunjula bya mugaso?,test
7666,some problems are just for a while.,ebizibu ebimu bya kaseera buseera.,test
10701,artists are earning less from music which make...,abayimbi bafuna kitono mu kuyimba ekibaletera ...,test
13538,the health summit will enforce strategies in o...,olukungaana lw'ebyobulamu lujja kubiriza emite...,test
8357,which smartphone did you buy?,ssimu ya sseereza ki gye wagula?,test


In [ ]:
df_train = df[df.split=='train'].copy()
df_dev = df[df.split=='dev'].copy()   #Do we need it?
df_test = df[df.split=='test'].copy()

In [ ]:
#downloading pretrained model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
import re

def word_tokenize(text):
    """
    Split a text into words, numbers, and punctuation marks
    (for languages where words are separated by spaces)
    """
    return re.findall('(\w+|[^\w\s])', text)

smpl = df_train.sample(7510, random_state=1)
smpl['lug_toks'] = smpl.Luganda.apply(tokenizer.tokenize)
smpl['eng_toks'] = smpl.English.apply(tokenizer.tokenize)
smpl['lug_words'] = smpl.Luganda.apply(word_tokenize)
smpl['eng_words'] = smpl.English.apply(word_tokenize)

In [ ]:
smpl.head()

,English,Luganda,split,lug_toks,eng_toks,lug_words,eng_words
6005,the country head is very worried about the loc...,pulezidenti mweraliikirivu nnyo olw'okulumbibw...,train,"[▁pu, lezidenti, ▁mw, erali, ikiri, vu, ▁nnyo,...","[▁the, ▁country, ▁head, ▁is, ▁very, ▁worried, ...","[pulezidenti, mweraliikirivu, nnyo, olw, ', ok...","[the, country, head, is, very, worried, about,..."
3565,managers need to bridge the gap between them ...,bamaneja beetaaga okuggyawo ebbanga eriri waka...,train,"[▁bam, ane, ja, ▁be, etaaga, ▁oku, gg, yawo, ▁...","[▁mana, gers, ▁need, ▁to, ▁bridge, ▁the, ▁gap,...","[bamaneja, beetaaga, okuggyawo, ebbanga, eriri...","[managers, need, to, bridge, the, gap, between..."
2106,"faithful people deserve to be rewarded, such a...","abantu abamazima basaana okusiimibwa, ng'abaku...",train,"[▁abantu, ▁ab, amazima, ▁basa, ana, ▁okusi, im...","[▁faithful, ▁people, ▁des, erve, ▁to, ▁be, ▁re...","[abantu, abamazima, basaana, okusiimibwa, ,, n...","[faithful, people, deserve, to, be, rewarded, ..."
299,the police barracks in nebbi got burnt.,enkambi ya poliisi mu nebbi yakutte omuliro.,train,"[▁enk, ambi, ▁ya, ▁poliisi, ▁mu, ▁neb, bi, ▁ya...","[▁the, ▁police, ▁barra, cks, ▁in, ▁neb, bi, ▁g...","[enkambi, ya, poliisi, mu, nebbi, yakutte, omu...","[the, police, barracks, in, nebbi, got, burnt, .]"
6891,older people have equal rights like other people.,abantu abakulu balina eddembe lye limu n'eryab...,train,"[▁abantu, ▁abak, ulu, ▁balina, ▁edd, embe, ▁ly...","[▁older, ▁people, ▁have, ▁equal, ▁rights, ▁lik...","[abantu, abakulu, balina, eddembe, lye, limu, ...","[older, people, have, equal, rights, like, oth..."


In [ ]:
stats = smpl[
    ['lug_toks', 'eng_toks', 'lug_words', 'eng_words']
].applymap(len).describe()
print(stats.lug_toks['mean'] / stats.lug_words['mean'])
print(stats.eng_toks['mean'] / stats.eng_words['mean'])
stats

1.6793003754153304
1.1746945504340776


,lug_toks,eng_toks,lug_words,eng_words
count,7510.000000,7510.000000,7510.000000,7510.000000
mean,15.545939,11.765246,9.257390,10.015579
std,5.428277,3.414483,3.294516,2.800443
min,3.000000,4.000000,2.000000,4.000000
25%,12.000000,9.000000,7.000000,8.000000
50%,15.000000,11.000000,9.000000,10.000000
75%,19.000000,14.000000,11.000000,12.000000
max,43.000000,30.000000,28.000000,23.000000


In [ ]:
from tqdm.auto import tqdm, trange
texts_with_unk = [
    text for text in tqdm(df.Luganda)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))

  0%|          | 0/15021 [00:00<?, ?it/s]

0


In [ ]:
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
model.cuda();
optimizer = Adafactor(
    [p for p in model.parameters() if p.requires_grad],
    scale_parameter=False,
    relative_step=False,
    lr=1e-4,
    clip_threshold=1.0,
    weight_decay=1e-3,
)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=1000)

In [ ]:
#TODO: Figure out what this cell does from the tutorial and change if we want

import random
LANGS = [('Luganda', 'lug'), ('English', 'eng')]

def get_batch_pairs(batch_size, data=df_train):
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append((item[l1]))
        yy.append((item[l2]))
    return xx, yy, long1, long2

print(get_batch_pairs(1))

(['obwavu bwe businze okuviirako abantu okutunda ente.'], ['poverty is the leading cause of people selling the cows.'], 'lug', 'eng')


In [ ]:
import gc
import torch

def cleanup():
    """Try to free GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
#TODO: Change these training parameters

batch_size = 12  # 32 already doesn't fit well to 15GB of GPU memory
max_length = 100  # token sequences will be truncated
training_steps = 1000  # Usually, I set a large number of steps,
# and then just interrupt the training manually
losses = []  # with this list, I do very simple tracking of average loss
MODEL_SAVE_PATH = '/gd/MyDrive/models/nllb-rus-tyv-v1'  # on my Google drive

In [ ]:
model.train()
x, y, loss = None, None, None
cleanup()

tq = trange(len(losses), training_steps)
for i in tq:
    xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        tokenizer.src_lang = lang2
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        # -100 is a magic value ignored in the loss function
        # because we don't want the model to learn to predict padding ids
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100

        loss = model(**x, labels=y.input_ids).loss
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

    except RuntimeError as e:  # usually, it is out-of-memory
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        cleanup()
        print('error', max(len(s) for s in xx + yy), e)
        continue

    if i % 1000 == 0:
        # each 1000 steps, I report average loss at these steps
        print(i, np.mean(losses[-1000:]))
        print(i, np.mean(losses[-1000:][0]))

    if i % 1000 == 0 and i > 0:
        model.save_pretrained(MODEL_SAVE_PATH)
        tokenizer.save_pretrained(MODEL_SAVE_PATH)

  0%|          | 0/1000 [00:00<?, ?it/s]

0 6.374882221221924
0 6.374882221221924


In [ ]:
def translate(
    text, src_lang='eng', tgt_lang='lug',
    a=32, b=3, max_input_length=1024, num_beams=6, **kwargs
):
    """Turn a text or a list of texts into a list of translations"""
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(
        text, return_tensors='pt', padding=True, truncation=True,
        max_length=max_input_length
    )
    model.eval() # turn off training mode
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams, **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

# Example usage:
a = 'All refugees were requested to register with the chairman.'
print(translate(a))
b = 'The border districts are insecure.'
print(translate(b))
c = 'More development is achieved when we work together.'
print(translate(c))
d = 'The school bus cannot accommodate most of the students.'
print(translate(d))
e = 'Men should support their women at all times.'
print(translate(e))

['lug abanoonyiboobubudamu bonna baasabiddwa okwewandiisa ne ssentebe.']
["lug disitulikiti z'oku nsalo tezilina bukuumi."]
['lug enkulaakulana endala etuuka ku bwe tukolera wamu.']
["lug bbaasi y'essomero tezisobola kusitula bayizi abasinga obungi."]
['lug abasajja balina okuyamba abakyala baabwe mu biseera byonna.']
